In [2]:
import sys
import time

import h5py as h5
import numpy as np
import matplotlib.pyplot as plt

sys.path.append(r"C:/Users/haoyuan/Documents/GitHub/CrystalDiff")

from CrystalDiff import util, pulse, lclsutil

In [3]:
energy_center = 10.
pre_length = 1e6

# Set up the pulse
my_pulse = pulse.GaussianPulse3D()
my_pulse.set_pulse_properties(central_energy=energy_center,
                              polar=[0., 1., 0.],
                              sigma_x=708.2581446128465,
                              sigma_y=708.2581446128465,
                              sigma_z=1.,
                              x0=np.array([0., 0., -pre_length]))

#
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#                  Get the momentum mesh
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
number_x = 250
number_y = 250
number_z = 10 ** 5
kx_grid, ky_grid, kz_grid, axis_info = lclsutil.get_k_mesh_3d(number_x=number_x,
                                                              number_y=number_y,
                                                              number_z=number_z,
                                                              delta_e_x=1e-4,
                                                              delta_e_y=1e-4,
                                                              delta_e_z=1e-3 / util.c)
kz_grid += my_pulse.klen0

# Apply fft shift
kx_grid = np.ascontiguousarray(np.fft.fftshift(kx_grid))
ky_grid = np.ascontiguousarray(np.fft.fftshift(ky_grid))
kz_grid = np.ascontiguousarray(np.fft.fftshift(kz_grid))

In [4]:
axis_info.keys()

dict_keys(['x_range', 'x_idx', 'x_idx_tick', 'dkx', 'energy_grid_x', 'y_range', 'y_idx', 'y_idx_tick', 'dky', 'energy_grid_y', 'z_range', 'z_idx', 'z_idx_tick', 'dkz', 'energy_grid_z', 'z_time_idx', 'z_time_tick', 'de_x_in_meV'])

# Simulation result

In [5]:
# Resultion from the h5 file
sum_field_from_h5 = 1.7966685312735459e+31

# Consider the integration unit
dx = axis_info['x_idx'][1] - axis_info['x_idx'][0]
dy = axis_info['y_idx'][1] - axis_info['y_idx'][0]
dz = axis_info['z_idx'][1] - axis_info['z_idx'][0]

# Simulation energy
simu_energy = sum_field_from_h5 * dx * dy * dz

print("The simulation pulse energy is {}".format(simu_energy))

The simulation pulse energy is 1.2832098422680755e+32


# Get the pulse energy

In [6]:
pulse_energy = 1e28 / np.sqrt(np.linalg.det(my_pulse.sigma_mat)) / np.power(2 * np.pi, 1.5)

print("The pulse energy is {}".format(pulse_energy))

The pulse energy is 4.697696268937501e+22


In [7]:
print(simu_energy / number_x / number_y / number_z / pulse_energy)

0.43705161638585194
